# Gallery Example: M/M/1 Queue with Feedback

This example demonstrates an M/M/1 queue with probabilistic feedback:
- **Arrivals**: Poisson process (Exponential inter-arrival times)
- **Service**: Exponential service times
- **Servers**: 1 server
- **Feedback**: Jobs may return to the queue with probability p
- **Scheduling**: FCFS (First-Come-First-Served)

With feedback probability p, the effective service rate becomes μ(1-p), increasing the utilization and response time.

In [ ]:
from line_solver import *
import numpy as np
GlobalConstants.set_verbose(VerboseLevel.STD)

In [ ]:
def gallery_mm1_feedback(p=1/3):    """Create M/M/1 queueing model with feedback        Parameters:    - p: Feedback probability (default 1/3)    """    model = Network('M/M/1-Feedback')        # Block 1: nodes    source = Source(model, 'Source')    queue = Queue(model, 'Queue', SchedStrategy.FCFS)    sink = Sink(model, 'Sink')        # Block 2: classes    oclass1 = OpenClass(model, 'Class1')    source.set_arrival(oclass1, Exp.fit_mean(1))  # λ = 1    queue.set_service(oclass1, Exp.fit_mean(0.5)) # μ = 2, mean service time = 0.5        # Block 3: topology - feedback routing    P = model.init_routing_matrix()    P.add_route(oclass1, source, queue, 1.0)  # All arrivals go to queue    P.add_route(oclass1, queue, queue, p)     # Feedback with probability p    P.add_route(oclass1, queue, sink, 1-p)    # Exit with probability 1-p    model.link(P)        return model# Create the model with default feedback probabilityp_feedback = 1/3model = gallery_mm1_feedback(p_feedback)print(f"Feedback probability: {p_feedback:.3f}")

## Theoretical Analysis for M/M/1 with Feedback

For M/M/1 with feedback probability p:
- **External arrival rate**: λ = 1
- **Service rate**: μ = 2
- **Effective service rate**: μ_eff = μ(1-p) = 2(1-1/3) = 4/3
- **Effective utilization**: ρ_eff = λ/μ_eff = 1/(4/3) = 0.75
- **Mean number of visits**: 1/(1-p) = 1/(1-1/3) = 1.5
- **Throughput**: X = λ = 1.0

In [ ]:
# Solve with multiple solvers
print("\n=== Solver Results ===")

# MVA Solver
solver_mva = MVA(model)
avg_table_mva = solver_mva.avg_table()
print("\nMVA Solver:")
print(avg_table_mva)

# CTMC Solver
solver_ctmc = CTMC(model, cutoff=20)
avg_table_ctmc = solver_ctmc.avg_table()
print("\nCTMC Solver:")
print(avg_table_ctmc)

# Fluid Solver
solver_fluid = FLD(model)
avg_table_fluid = solver_fluid.avg_table()
print("\nFluid Solver:")
print(avg_table_fluid)

In [ ]:
# Compare with different feedback probabilities
print("\n=== Impact of Feedback Probability ===")
p_values = [0.0, 0.2, 0.4, 0.6, 0.8]

for p in p_values:
    model_p = gallery_mm1_feedback(p)
    solver = MVA(model_p)
    avg_table = solver.avg_table()
    
    # Extract utilization and response time from results
    util = float(avg_table.iloc[1, 1])  # Queue utilization
    resp_time = float(avg_table.iloc[1, 2])  # Queue response time
    
    print(f"p={p:.1f}: Utilization={util:.3f}, Response Time={resp_time:.3f}")